In [1]:
from tensorflow.keras import datasets
import tensorflow as tf
import matplotlib.pyplot as plt


In [6]:
# 데이터 로딩
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]


In [7]:
X_train = X_train/255.0
X_valid = X_valid/255.0
X_test = X_test/255.0


In [10]:
from functools import partial

# partial : 파라메터만 미리 정해준 또 다른 함수를 생성

DefaultConv2D = partial(tf.keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

# 이미지가 아주 크지 않아서 64개의 큰 필터 (7 x 7)와 스트라이드 1을 사용

model = tf.keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    # 풀링 크기가 2인 최대 풀링 층을 추가하여 공간 방향 차원을 절반으로 줄인다.
    tf.keras.layers.MaxPooling2D(pool_size=2),
    # 동일한 구조를 두번 반복, 이미지가 클 때는 이러한 구조를 더 많이 반복이 가능함
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    # CNN이 출력층에 다다를수록 필터 개수가 늘어남 => 저수준(작은 동심원, 수평선)의 개수는 적지만 이를 연결하여 고수준 특성을 만든다.
    # 풀링층 다음에 필터수를 두배로 늘린다(풀링층에서 줄어들었으므로)
    # 풀링층이 공간 방향 차원을 반으로 줄이므로 이어지는 층에서 특성 맵 개수를 두배로 늘린다
    # 밀집 네트워크는 샘플의 특성으로 1차원 배열을 기대하므로 입력을 일렬로 펼쳐야 함.
    tf.keras.layers.Flatten(),
    # 두 개의 은닉층과 하나의 출력층으로 구성된 완전 연결층
    tf.keras.layers.Dense(units=128, activation='relu'),
    # 밀집 층 사이에 과대적합을 줄이기 위해 50%의 드롭아웃 비율을 가진 드롭아웃 층 추가
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation='softmax'),
])


In [12]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))


Epoch 1/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.7643 - accuracy: 0.7228 - val_loss: 0.3953 - val_accuracy: 0.8490
Epoch 2/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4250 - accuracy: 0.8531 - val_loss: 0.3473 - val_accuracy: 0.8834
Epoch 3/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.3617 - accuracy: 0.8776 - val_loss: 0.2975 - val_accuracy: 0.8896
Epoch 4/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.3292 - accuracy: 0.8869 - val_loss: 0.2624 - val_accuracy: 0.9028
Epoch 5/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.2992 - accuracy: 0.8990 - val_loss: 0.2448 - val_accuracy: 0.9124
Epoch 6/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.2740 - accuracy: 0.9044 - val_loss: 0.2747 - val_accuracy: 0.9002
Epoch 7/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.2602 - accuracy: 0.9098 - val_loss: 0.2613 - val_ac

In [13]:
score = model.evaluate(X_test, y_test)  # loss, accuracy
score

313/313 [==============================] - 1s 3ms/step - loss: 0.2896 - accuracy: 0.9005


[0.2895979881286621, 0.9004999995231628]